The orginal "500 Cities: Coronary heart disease among adults aged >=18 years" dataset from the cdc includes estimates for the 500 largest US cities and approximately 28,000 census tracts within these cities. These estimates can be used to identify emerging health problems and to inform development and implementation of effective, targeted public health prevention activities. 

https://data.cdc.gov/500-Cities-Places/500-Cities-Coronary-heart-disease-among-adults-age/cqcq-r6f8

In [1]:
# Dependencies
import pandas as pd
from pathlib import Path

In [2]:
# Name of the CSV file
file = Path('Resources/500_Cities_Coronary_heart_disease_among_adults.csv')

In [3]:
# Read the CSV in pandas
heart_disease_df = pd.read_csv(file)

In [4]:
heart_disease_df.columns


Index(['Year ', 'StateAbbr ', 'StateDesc     ', 'CityName                   ',
       'GeographicLevel ', 'DataSource ', 'Category        ',
       'UniqueID            ',
       'Measure                                             ',
       'Data_Value_Unit ', 'DataValueTypeID ', 'Data_Value_Type         ',
       'Data_Value ', 'Low_Confidence_Limit ', 'High_Confidence_Limit ',
       'Data_Value_Footnote_Symbol ',
       'Data_Value_Footnote                              ', 'PopulationCount ',
       'GeoLocation                       ', 'CategoryID ', 'MeasureId ',
       'CityFIPS ', 'TractFIPS   ', 'Short_Question_Text'],
      dtype='object')

In [5]:
# Remove unnecessary columns - create a new data frame with only the columns we need

heart_disease_df = heart_disease_df[['StateDesc     ', 'Data_Value ', 'PopulationCount ',
       'GeoLocation                       ']]

heart_disease_df.head()


,StateDesc,Data_Value,PopulationCount,GeoLocation
0,California,4.8,"144,186","(37.6329591551, -122.077051051)"
1,Alabama,6.7,"180,105","(34.6989692671, -86.6387042882)"
2,Arizona,3.8,"236,123","(33.28318981, -111.852210033)"
3,Arizona,5.9,"117,517","(33.6803835177, -112.452218439)"
4,California,5.3,"76,616","(33.8880417923, -118.127100236)"


In [6]:
# Rename the columns for better understanding and reorder the columns 
# Data Value - clarify percentage unit

heart_disease_data_df = heart_disease_df.rename(columns={'StateDesc     ': "State",
                                                         'Data_Value ': "Percent of Population (%)", 'PopulationCount ': "Population",
                                                         'GeoLocation                       ': "Geo Location"})

heart_disease_data_df.head()

,State,Percent of Population (%),Population,Geo Location
0,California,4.8,"144,186","(37.6329591551, -122.077051051)"
1,Alabama,6.7,"180,105","(34.6989692671, -86.6387042882)"
2,Arizona,3.8,"236,123","(33.28318981, -111.852210033)"
3,Arizona,5.9,"117,517","(33.6803835177, -112.452218439)"
4,California,5.3,"76,616","(33.8880417923, -118.127100236)"


In [7]:
# Groupby state
heart_disease_data_df = heart_disease_data_df.fillna(0)
heart_disease_data_df

#Set State as the index
organized_df = heart_disease_data_df.set_index("State")
organized_df

# Sort the index so the states are grouped in alphabetical order
organized_heart_disease_df = organized_df.sort_index()
organized_heart_disease_df

# Identify "United States" rows and remove from df because they're not needed
partial_match_row = organized_heart_disease_df[organized_heart_disease_df.index.str.contains('United', case=False)]
partial_match_row

organized_heart_disease_df = organized_heart_disease_df.drop(partial_match_row.index, axis=0)
organized_heart_disease_df



,Percent of Population (%),Population,Geo Location
State,,,
Alabama,5.7,"1,708","(30.6919609185, -88.0738103038)"
Alabama,3.3,"1,724","(33.1650878227, -87.5617780419)"
Alabama,4.8,815,"(33.6718848706, -86.6772510465)"
Alabama,10.1,"1,605","(32.3720175299, -86.3087966777)"
Alabama,5.9,"4,223","(33.4019456118, -86.8333493253)"
...,...,...,...
Wyoming,5.8,"4,580","(41.1243595766, -104.813946446)"
Wyoming,5.7,"4,331","(41.1347650805, -104.819326156)"
Wyoming,5.9,"3,026","(41.1414159833, -104.829095061)"


In [16]:
# Clean the data within each column - remove spaces and separate geolocation string

# Remove leading and trailing spaces from the values in columns

organized_heart_disease_df[['Percent of Population (%)', 'Population', 'Geo Location']] = organized_heart_disease_df[['Percent of Population (%)', 'Population', 'Geo Location']].apply(lambda x: x.str.strip())

# Remove nondigit values like commas and parenthesis from each column

organized_heart_disease_df['Population'] = organized_heart_disease_df['Population'].str.replace(r'[,]', '', regex=True)


organized_heart_disease_df['Geo Location'] = organized_heart_disease_df['Geo Location'].str.replace(r'[()]', '', regex=True)

# Separate geolocation string into Latitude and Longitude columns 
organized_heart_disease_df[['Latitude', 'Longitude']] = organized_heart_disease_df['Geo Location'].str.split(', ', expand=True)

# Remove the original Geo Location column

del organized_heart_disease_df['Geo Location']

organized_heart_disease_df


,Percent of Population (%),Population,Latitude,Longitude
State,,,,
Alabama,5.7,1708,30.6919609185,-88.0738103038
Alabama,3.3,1724,33.1650878227,-87.5617780419
Alabama,4.8,815,33.6718848706,-86.6772510465
Alabama,10.1,1605,32.3720175299,-86.3087966777
Alabama,5.9,4223,33.4019456118,-86.8333493253
...,...,...,...,...
Wyoming,5.8,4580,41.1243595766,-104.813946446
Wyoming,5.7,4331,41.1347650805,-104.819326156
Wyoming,5.9,3026,41.1414159833,-104.829095061


In [17]:
# Convert all columns to integers 
columns_to_convert = ["Percent of Population (%)", "Population", "Latitude", "Longitude"]
organized_heart_disease_df[["Percent of Population (%)", "Population", "Latitude", "Longitude"]] = organized_heart_disease_df[["Percent of Population (%)", "Population", "Latitude", "Longitude"]].apply(pd.to_numeric)

organized_heart_disease_df.dtypes


Percent of Population (%)    float64
Population                     int64
Latitude                     float64
Longitude                    float64
dtype: object

In [18]:
# Convert new organized_heart_disease_df to a csv file for future use

organized_heart_disease_df.to_csv("Resources/coronary_heart_disease_by_state.csv")

In [ ]:
#merge with coffee data frame

In [11]:
#Creat an empty list for states and use a for loop to add the states to the list (might not be all fifty states)

states = []


#inside the for loop (nested for loop) add the pop% to the list value(state index)
#calculate the mean of pop % with heart disease for each state

In [12]:
# create a new dataframe with the mean of each state's pop% and population - excluding the lat and lon for now
#merge with coffee df from site
# iloc on only the states we need to look at (Michigan, Delaware, Utah, Arizona)

#create an arcgis map to show coffee and heart disease